In [1]:
!pip install transformers==3.0.2
!pip install datasets
!pip install evaluate
!pip install tensorflow
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.0/769.0 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=23a6442adcdf430fa718aeb2d6b14af823c4ff6a8e90fac26bd0b1d1441e5a11
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 12.8

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from datasets import load_dataset, DatasetDict, load_metric, load_from_disk, Dataset

import pandas as pd

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Models/FiD

/content/drive/MyDrive/Models/FiD


In [4]:
df = pd.read_excel('/content/WOT_internal_refined_4.xlsx')
df['comment'] = df['comment'].fillna('')

# remove external Question pairs
df['not external'] = df['comment'].apply(lambda x: 'external' not in x.split(' + '))
df = df[df['not external']]

df = df[['domain', 'data_split', 'question', 'history','title', 'steps', 'ingredients', 'description', 'Context', 'answer_extr', 'answer_start', 'number_answer_elements', 'comment']].rename(columns={'Context': 'context', 'answer_extr': 'answers'})

df.to_csv('/content/temp.csv')
df = pd.read_csv('/content/temp.csv').rename(columns={'Unnamed: 0': 'id'})
df['id'] = df['id'].astype(str)

df['answers'] = df['answers'].apply(lambda x: x[2:-2].split("\', \'"))
df['answer_start'] = df['answer_start'].apply(lambda x: x[1:-1].split(", "))
df['answer_start'] = df['answer_start'].apply(lambda x: [-1] if x[0]=='' else x)
df['answer_start'] = df['answer_start'].apply(lambda x: [int(el) for el in x])
df['answers'] = df.apply(lambda row: {'text': row['answers'], 'answer_start': row['answer_start']} , axis=1)

In [5]:
df['input'] = df.apply(lambda row: f"{row['question']}SPLITInstructions: {row['context']}\nHistory: {row['history']}\n", axis=1)

In [6]:
df_1_span = df[df['number_answer_elements'] == 1]

# FiD specific
df_1_span['answers'] = df_1_span['answers'].apply(lambda x: x['text'])
df_1_span['history'] = df_1_span['history'].fillna(' ')
df_1_span['steps'] = df_1_span['steps'].apply(lambda x: x[2:-2].split("\', \'"))

df_1_span['ctxs1'] = df_1_span.apply(lambda row: [{'title': 'conversation history', 'text': row['history']}], axis = 1)
df_1_span['ctxs1'] = df_1_span.apply(lambda row: row['ctxs1'] + [{'title': f'{row["title"]} description', 'text': row['description']}] , axis = 1)
df_1_span['ctxs1'] = df_1_span.apply(lambda row: row['ctxs1'] + [{'title': f'{row["title"]} ingredients', 'text': row['ingredients']}] , axis = 1)
df_1_span['ctxs1'] = df_1_span.apply(lambda row: row['ctxs1'] + [{'title': f'{row["title"]} Step {i+1}', 'text': row['steps'][i]} for i in range(len(row['steps']))] , axis = 1)


def split_context(context):
    context = context.split(' ')
    start_points = list(range(0,len(context), 200))
    end_points = list(range(250,len(context), 200))
    out = [context[start_points[i]:end_points[i]] for i in range(len(end_points))]
    #append proper ending sequence
    out.append(context[-250:])
    out = [' '.join(el) for el in out]
    return out

df_1_span['ctxs2'] = df_1_span.apply(lambda row: f"{row['context']}\nHistory: {row['history']}\n", axis=1)
df_1_span['ctxs2'] = df_1_span['ctxs2'].apply(split_context)
df_1_span['ctxs2'] = df_1_span.apply(lambda row: [{'title': f'{row["title"]} snippet {i}', 'text': row['ctxs2'][i]} for i in range(len(row['ctxs2']))] , axis = 1)

df_1_span['ctxs'] = df_1_span['ctxs1']

df_1_span = df_1_span[['id', 'domain', 'data_split', 'comment', 'question', 'answers', 'ctxs']].reset_index()

ds_1_span_train = Dataset.from_pandas(df_1_span[df_1_span['data_split']=='train'].drop(columns=['data_split']))
ds_1_span_test = Dataset.from_pandas(df_1_span[df_1_span['data_split']=='test'].drop(columns=['data_split']))
ds_1_span_val = Dataset.from_pandas(df_1_span[df_1_span['data_split']=='validation'].drop(columns=['data_split']))

ds_1_span = DatasetDict({'train': ds_1_span_train,
                         'test': ds_1_span_test,
                         'validation': ds_1_span_val})
print(ds_1_span)

<ipython-input-6-ad2dcf6ab156>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_span['answers'] = df_1_span['answers'].apply(lambda x: x['text'])
<ipython-input-6-ad2dcf6ab156>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_span['history'] = df_1_span['history'].fillna(' ')
<ipython-input-6-ad2dcf6ab156>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

DatasetDict({
    train: Dataset({
        features: ['index', 'id', 'domain', 'comment', 'question', 'answers', 'ctxs', '__index_level_0__'],
        num_rows: 661
    })
    test: Dataset({
        features: ['index', 'id', 'domain', 'comment', 'question', 'answers', 'ctxs', '__index_level_0__'],
        num_rows: 64
    })
    validation: Dataset({
        features: ['index', 'id', 'domain', 'comment', 'question', 'answers', 'ctxs', '__index_level_0__'],
        num_rows: 72
    })
})


In [7]:
ds_1_span['train'].to_json("/content/WoT_extract_train.jsonl")
ds_1_span['test'].to_json("/content/WoT_extract_test.jsonl")
ds_1_span['validation'].to_json("/content/WoT_extract_eval.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

355190

## Untuned

In [ ]:
! python test_reader.py \
        --model_path /content/drive/MyDrive/Models/FiD/pretrained_models/nq_reader_base \
        --eval_data /content/WoT_extract_test.jsonl \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --name WoT_extractive_0Shot \
        --checkpoint_dir checkpoint

2023-02-27 16:16:21.409709: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 16:16:22.797890: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:16:22.797997: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:16:22.798017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

## Finetune

In [ ]:
! python train_reader.py \
        --train_data /content/WoT_extract_train.jsonl \
        --eval_data /content/WoT_extract_eval.jsonl \
        --model_size base \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --total_step 5000 \
        --warmup_step 500 \
        --name WoT_extractive_finetune \
        --checkpoint_dir /content/drive/MyDrive/Models/FiD/pretrained_models/nq_reader_base


2023-02-27 16:22:18.488131: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 16:22:19.365526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:22:19.365630: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:22:19.365649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [ ]:
! python test_reader.py \
        --model_path /content/drive/MyDrive/Models/FiD/pretrained_models/nq_reader_base/WoT_extractive_finetune/checkpoint/best_dev \
        --eval_data /content/WoT_extract_test.jsonl \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --name WoT_extractive_finetuned \
        --checkpoint_dir checkpoint

2023-02-27 16:22:25.892716: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 16:22:27.144248: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:22:27.144358: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:22:27.144379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

## Few Shot Experiment

In [11]:
! python train_reader.py \
        --train_data /content/WoT_extract_test.jsonl \
        --eval_data /content/WoT_extract_eval.jsonl \
        --model_size base \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --total_step 5000 \
        --warmup_step 500 \
        --name WoT_extractive_finetune \
        --checkpoint_dir /content/drive/MyDrive/Models/FiD/pretrained_models/nq_reader_base_few_shot

2023-02-28 23:08:12.020264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 23:08:13.343918: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 23:08:13.344078: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 23:08:13.344102: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

### Tuned

In [12]:
! python test_reader.py \
        --model_path /content/drive/MyDrive/Models/FiD/pretrained_models/nq_reader_base_few_shot/WoT_extractive_finetune/checkpoint/best_dev \
        --eval_data /content/WoT_extract_train.jsonl \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --name WoT_extractive_finetuned \
        --checkpoint_dir checkpoint

2023-02-28 23:29:18.534971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 23:29:19.484607: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 23:29:19.484765: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 23:29:19.484786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

### Untuned

In [ ]:
! python test_reader.py \
        --model_path /content/drive/MyDrive/Models/FiD/pretrained_models/nq_reader_base \
        --eval_data /content/WoT_extract_train.jsonl \
        --per_gpu_batch_size 1 \
        --n_context 10 \
        --name WoT_extractive_0Shot \
        --checkpoint_dir checkpoint

2023-02-27 16:28:03.665262: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 16:28:04.582474: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:28:04.582591: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 16:28:04.582611: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War